In [1]:
import sys
from transformers import AutoModelForCausalLM, AutoTokenizer
sys.path.append('../')
from utils.data_utils import prepare_data,load_dataset_config,format_instruction_prompt
import torch

/home/infidea/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [78]:
def change_str(example):
    change_map = {
        "①": "A.",
        "②": "B.",
        "③": "C.",
        "④": "D."
    }
    for key, value in change_map.items():
        example['question'] = example['question'].replace(key, value)
        example['answer'] = example['answer'].replace(key, value)
    return example

In [79]:
dataset_new = dataset.map(change_str)

Map: 100%|██████████| 98/98 [00:00<00:00, 7318.86 examples/s]


In [27]:
from datasets import load_dataset

tesat = load_dataset("overfit-brothers/TESAT")

Generating train split: 138 examples [00:00, 18033.27 examples/s]


In [31]:
tesat['train'][2]

{'question': '종합부동산세의 존치와 폐지를 주장하는 다음의 글을 읽고 이들 논리로부터 유추한 보기의 설명 중 가장 부적절한 것을 고르시오.\n\n◆존치론=종합부동산세는 소수의 부동산 부자들로부터 세금을 걷어 지방의 서민들을 지원하는 정의로운 세금이다. 미국 등 선진국은 부동산 보유세율이 우리나라보다 몇 배나 높다. 보유세율을 정상화한다는 의미에서도 종부세는 폐지할 수 없다. 부동산 시장을 안정시키고 투기를 잡기 위해서라도 종부세는 필수적이다. 종부세를 폐지하는 것은 소수의 부자를 위한 것이다.\n\n◆폐지론=선진국의 보유세는 대부분 종부세와 같은 누진율이 아닌 정률세로 운영된다. 부동산을 가진 모든 국민이 동일한 세율로 납부하는 것이다. 일부 부자에게 지방재정에 대한 책임을 떠넘긴다는 면에서 이 세금은 정의롭지도 못하다. 더구나 고가의 부동산을 보유했다고 해서 진짜 부자인 것도 아니다. 세금이 부동산 가격을 안정시키는 효과도 없다.\n\n1 존치론자의 논리대로라면 종부세가 아닌 재산세를 올려야 한다.\n2 폐지론자는 세금은 고루 부담하는 보편성을 가져야 정의롭다고 본다.\n3 폐지론자는 순자산에 매기는 부유세에 찬성할 가능성이 있다.\n4 존치론자는 세금 인상이 부동산 가격을 안정시키는 기능을 한다고 믿는다.\n5 폐지론자는 보유세를 무겁게 매기는 것에 포괄적으로 반대한다.',
 'answer': "폐지론자는 부유세에 찬성할 가능성이 있다'는 3번 선택지를 정답으로 고른 사람은 폐지론의 논거중 '고가 부동산을 보유했다고 해서 진짜 부자인 것도 아니다'는 부분을 흘려 읽었기 때문이다.\n\n문제를 정확하게 읽지 않았기 때문에 '종부세 반대는 곧 부자들을 옹호하는 것'이라고 쉽게 결론짓게 되고 결과적으로 3번을 고르게 되었다. 주장을 비판적으로 읽는 노력이 필요한 것은 바로 이 때문이다.\n\n주장의 내용을 구체적으로 살펴봐야 하는 많은 사회적 쟁점들을 단순히 이념적 편가르기 식으로 받아들이고 만다면 진정한 토론이 성립되기 힘들다. 종부세를 찬성하면 가

In [1]:
import sys
from transformers import AutoModelForCausalLM, AutoTokenizer
sys.path.append('../')
from utils.data_utils import prepare_data,load_dataset_config,format_instruction_prompt
import torch

/home/infidea/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model2 = "outputs/Qwen2.5-7B-Instruct-YS-JJIN-FINAL"
model2 = AutoModelForCausalLM.from_pretrained(model2, trust_remote_code=True, device_map="auto", torch_dtype=torch.bfloat16)


/home/infidea/.local/lib/python3.8/site-packages/torchao/ops.py:12: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  return torch.library.impl_abstract(f"{name}")(func)
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.39it/s]


In [4]:
model_name = "mergekit/merged"
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map="auto", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.37it/s]


In [17]:
chat = {"role":"user", "content":"hello"}
tem = tokenizer.apply_chat_template([chat], tokenize=False)
tem

'<|im_start|>system\nYou are a helpful assistant.\nPlease follow these steps:\n1. Carefully work through a step-by-step solution to the question.\n2. After completing the solution, state the final answer.<|im_end|>\n<|im_start|>user\nhello\n따라서 정답은<|im_end|>\n<|im_start|>assistant\n'

In [ ]:
model.push_to_hub('overfit-brothers/Qwen2.5-7B-Instruct-YS-JJIN-FINAL', private=True, token="hf_XXX")
tokenizer.push_to_hub('overfit-brothers/Qwen2.5-7B-Instruct-YS-JJIN-FINAL', private=True, token="hf_XXX")

In [5]:
PROMPT_FORMAT = """### 질문: {} ### 정답: """

def make_chat(text):
    prompt = PROMPT_FORMAT.format(text)
    return prompt

def make_answer(model,tokenizer, instruction):

    chat_template = make_chat(instruction)
    inputs = tokenizer(chat_template, return_tensors="pt").to('cuda')
    answer = model.generate(**inputs, max_new_tokens = 1024, use_cache = True, repetition_penalty = 1.2)
    answer_chat = tokenizer.decode(answer[0], skip_special_tokens=True)
    print(answer_chat)
    return answer_chat
# 답 C
q = """
아래와 같은 경기변동의 양상이 나타날 때 필요한 정책은?
- 지난해보다 실업률 급격히 상승
- 국내총생산(GDP)은 지난해보다 감소
- 소비자물가지수가 지난해보다 하락해 마이너스를 기록
### 선택지:
A. 세율을 인상하고 통화량을 늘린다.
B. 통화량을 늘리고 이자율을 높인다.
C. 이자율을 낮추고 세율을 인하한다.
D. 정부 지출을 줄이고 통화량을 줄인다.
E. 정부 지출을 줄이고 이자율을 낮춘다.
"""
# 답 B
q2 = """아래 나열한 지표와 가장 관련이 깊은 것은 무엇인가?
- LTV  - DTI  - DSR
### 선택지:
A. 기업손실     B. 가계부채     C. 재정적자     D. 무역수지     E. 환율변동"""

# 답 A/home/infidea/workspace/magpie
q3 = """주어진 데이터프레임을 보고 질문에 요구에 알맞는 코드를 선택하시요.
### df.head()
|    | Symbol    | Series   | Date        |   Prev Close |   Open Price |   High Price |   Low Price |   Last Price |   Close Price |   Average Price |   Total Traded Quantity |    Turnover |   No. of Trades |   Deliverable Qty |   % Dly Qt to Traded Qty |
|---:|:----------|:---------|:------------|-------------:|-------------:|-------------:|------------:|-------------:|--------------:|----------------:|------------------------:|------------:|----------------:|------------------:|-------------------------:|
|  0 | GODREJIND | EQ       | 15-May-2017 |       564.6  |       581    |       584    |      568.5  |       578.9  |        578.55 |          578.09 |                  797171 | 4.60836e+08 |           21649 |            360927 |                    45.28 |
|  1 | GODREJIND | EQ       | 16-May-2017 |       578.55 |       581.45 |       589    |      572.25 |       583.8  |        584.8  |          583.6  |                  500223 | 2.9193e+08  |           17204 |            210364 |                    42.05 |
|  2 | GODREJIND | EQ       | 17-May-2017 |       584.8  |       583    |       594    |      576.85 |       584.9  |        588.6  |          588.74 |                  504155 | 2.96815e+08 |            8567 |            261667 |                    51.9  |
|  3 | GODREJIND | EQ       | 18-May-2017 |       588.6  |       582    |       588.85 |      571.2  |       572.25 |        574.6  |          580.9  |                  223583 | 1.29879e+08 |            7144 |             99785 |                    44.63 |
|  4 | GODREJIND | EQ       | 19-May-2017 |       574.6  |       581    |       585.8  |      567.55 |       579.85 |        578    |          577.31 |                  245436 | 1.41692e+08 |            4969 |             68041 |                    27.72 |

### 질문:
“거래량” 열의 중간값을 계산합니다.

### 선택지:
a) ```python
df['Total Traded Quantity'].median()
```
b) ```python
df['Total Traded Quantity'].mean()
```
c) ```python
df['Total_Traded_Quantity'].median()
```
d) ```python
df['Total Traded Quantity'].sum()
```
"""
# 답 B
q4 = """
정보 비대칭과 관련한 설명으로 옳지 않은 것은?
### 선택지: 
A. 노동시장에서 도덕적 해이를 해결하기 위해 효율성 임금을 지급한다.
B. 신호 보내기는 정보가 부족한 주체가 상대방의 정보를 얻기 위해 하는 행위이다.
C. 도덕적 해이는 사용자가 대리인의 행동을 완벽하게 감시할 수 없을 때 발생한다.
D. 정보 비대칭이 존재하는 시장에서는 정부의 개입으로 자원 배분의 효율성을 개선할 수도 있다.
E. 보험회사는 가입자의 자가 부담금이 있는 보험 상품을 판매함으로써 정보 비대칭으로 발생하는 문제를 완화할 수 있다.
"""
# 답 B
q5 = """
한 나라의 총수요를 증가시키는 요인이 _아닌_ 것은?
### 선택지 : 
A. 기업의 투자 증가 
B. 해외 직구 수입 증가
C. 정부의 재정지출 증가 
D. 가계의 소비지출 증가
E. 중앙은행의 기준금리 인하
"""


In [10]:
# merged
make_answer(model,tokenizer,q5)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### 질문: 
한 나라의 총수요를 증가시키는 요인이 _아닌_ 것은?
### 선택지 : 
A. 기업의 투자 증가 
B. 해외 직구 수입 증가
C. 정부의 재정지출 증가 
D. 가계의 소비지출 증가
E. 중앙은행의 기준금리 인하
 ### 정답:  B


'### 질문: \n한 나라의 총수요를 증가시키는 요인이 _아닌_ 것은?\n### 선택지 : \nA. 기업의 투자 증가 \nB. 해외 직구 수입 증가\nC. 정부의 재정지출 증가 \nD. 가계의 소비지출 증가\nE. 중앙은행의 기준금리 인하\n ### 정답:  B'

In [8]:
# JJIN_FINAL
make_answer(model2,tokenizer,q3)

### 질문: 주어진 데이터프레임을 보고 질문에 요구에 알맞는 코드를 선택하시요.
### df.head()
|    | Symbol    | Series   | Date        |   Prev Close |   Open Price |   High Price |   Low Price |   Last Price |   Close Price |   Average Price |   Total Traded Quantity |    Turnover |   No. of Trades |   Deliverable Qty |   % Dly Qt to Traded Qty |
|---:|:----------|:---------|:------------|-------------:|-------------:|-------------:|------------:|-------------:|--------------:|----------------:|------------------------:|------------:|----------------:|------------------:|-------------------------:|
|  0 | GODREJIND | EQ       | 15-May-2017 |       564.6  |       581    |       584    |      568.5  |       578.9  |        578.55 |          578.09 |                  797171 | 4.60836e+08 |           21649 |            360927 |                    45.28 |
|  1 | GODREJIND | EQ       | 16-May-2017 |       578.55 |       581.45 |       589    |      572.25 |       583.8  |        584.8  |          583.6  |                  

'### 질문: 주어진 데이터프레임을 보고 질문에 요구에 알맞는 코드를 선택하시요.\n### df.head()\n|    | Symbol    | Series   | Date        |   Prev Close |   Open Price |   High Price |   Low Price |   Last Price |   Close Price |   Average Price |   Total Traded Quantity |    Turnover |   No. of Trades |   Deliverable Qty |   % Dly Qt to Traded Qty |\n|---:|:----------|:---------|:------------|-------------:|-------------:|-------------:|------------:|-------------:|--------------:|----------------:|------------------------:|------------:|----------------:|------------------:|-------------------------:|\n|  0 | GODREJIND | EQ       | 15-May-2017 |       564.6  |       581    |       584    |      568.5  |       578.9  |        578.55 |          578.09 |                  797171 | 4.60836e+08 |           21649 |            360927 |                    45.28 |\n|  1 | GODREJIND | EQ       | 16-May-2017 |       578.55 |       581.45 |       589    |      572.25 |       583.8  |        584.8  |          583.6  |            

### MCQA:
  - "overfit-brothers/example-mcqa"
  - "overfit-brothers/TESAT"
  - "overfit-brothers/krx-textbook"
  - "overfit-brothers/mmmlu_revised_augmented_cot" 
  - "overfit-brothers/financial-market-law-mcqa"
### instruct:
  - "overfit-brothers/example-it"
  - "overfit-brothers/financial_judgement_instructions"
  - "overfit-brothers/financial-market-law-it"

In [1]:
from datasets import load_dataset
financial = load_dataset("overfit-brothers/financial-mmlu-ko-cot")
# financial.save_to_disk("financial_dataset.json")

/home/infidea/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
res_list = []
for datum in financial['train']:
    question = datum['question']
    answer = datum['answer']
    question = question.replace('\n1.', ' ### 선택지: A.').replace('\n2.',' B.').replace('\n3.',' C.').replace('\n4.',' D.').replace('\n5.',' E.')
    answer = answer.replace('풀이: ','')
    answer = answer.replace('1번','A').replace('2번','B').replace('3번','C').replace('4번','D').replace('5번','E')
    answer = answer.replace('1.','A.').replace('2.','B.').replace('3.','C.').replace('4.','D.').replace('5.','E.')
    answer = answer.replace('정답은','정답:')
    answer = answer.replace('정답: 1','### 정답: A').replace('정답: 2','### 정답: B').replace('정답: 3','### 정답: C').replace('정답: 4','### 정답: D').replace('정답: 5','### 정답: E')
    res = {
        'question': question,
        'answer': answer
    }
    res_list.append(res)

In [11]:
import json
with open('financial_revise.json','w',encoding='utf-8') as f:
    json.dump(res_list,f,ensure_ascii=False,indent=4)


In [19]:
financial['train'][-100]

{'question': '테일러 준칙에 따르면 중앙은행의 단기목표금리는 어떤 요소들을 고려하여 결정되는가?\n1. 물가갭(실제 인플레이션율 - 목표인플레이션율)과 생산갭(실제성장률 - 잠재성장률)\n2. 물가갭(실제 인플레이션율 - 목표인플레이션율)과 실업률\n3. 생산갭(실제성장률 - 잠재성장률)과 임금인상률\n4. 실업률과 임금인상률',
 'answer': '테일러 준칙은 중앙은행의 단기목표금리를 설정할 때 두 가지 주요 요소를 고려합니다. 첫째, 물가갭은 실제 인플레이션율과 목표 인플레이션율의 차이를 나타내며, 이는 인플레이션 조절을 위한 금리 조정의 필요성을 반영합니다. 둘째, 생산갭은 실제 성장률과 잠재 성장률의 차이를 나타내며, 이는 경제의 과열 또는 침체를 방지하기 위한 금리 조정의 필요성을 반영합니다. 따라서, 테일러 준칙에 따르면 중앙은행은 물가갭과 생산갭을 고려하여 단기목표금리를 결정합니다. 정답은 1입니다.'}

In [2]:
import json

with open('financial_revise.json', 'r',encoding='utf-8') as f:
    data = json.load(f)

In [5]:
data
new_list = []
for datum in data:
    question = ' '.join(datum['question'].replace('\n', ' ').split())
    answer = ' '.join(datum['answer'].replace('\n', ' ').split())
    if answer.endswith('입니다.'):
        answer = answer[:-4]
    new_list.append({'question': question, 'answer': answer})

In [7]:
new_list

with open('financial_revise.json','w',encoding='utf-8') as f:
    json.dump(new_list,f,ensure_ascii=False,indent=4)

In [3]:
len(data)

455

In [7]:
from datasets import load_dataset
label_dataset = load_dataset('allganize/financial-mmlu-ko')['test']

In [9]:
answer_map = {'1': 'A', '2': 'B', '3': 'C', '4': 'D', '5': 'E'}
label_list = []
for datum in label_dataset:
    conv = datum['conversations']
    answer = answer_map[conv[-1]['value']]
    label_list.append(answer)


In [12]:
total_len = len(data)
answer_cnt = 0
correct_list = []
for label, datum in zip(label_list, data):
    hypothesis_answer = max(['A', 'B', 'C', 'D', 'E'], key=lambda x: datum['answer'].count(x))
    if label == hypothesis_answer:
        answer_cnt += 1
        correct_list.append(datum)
print(f'{answer_cnt}/{total_len}')
    

414/455


In [14]:
from datasets import Dataset
correct_dataset = Dataset.from_list(correct_list)
correct_dataset.push_to_hub("overfit-brothers/financial-mmlu-ko-cot-revised")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.08s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/overfit-brothers/financial-mmlu-ko-cot-revised/commit/127e30c6c42160c6443fffac555bf0dbdb63e539', commit_message='Upload dataset', commit_description='', oid='127e30c6c42160c6443fffac555bf0dbdb63e539', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/overfit-brothers/financial-mmlu-ko-cot-revised', endpoint='https://huggingface.co', repo_type='dataset', repo_id='overfit-brothers/financial-mmlu-ko-cot-revised'), pr_revision=None, pr_num=None)